<a href="https://colab.research.google.com/github/jurados/AstroinformaticProject/blob/main/kbmod_stamps_access_w_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt

# KBMOD Coadd Stamps

## README

The following data was generated for use as part of the [KBMOD](https://github.com/dirac-institute/kbmod) project, for the purpose of creating a machine learning model to filter out false positve results from the KBMOD search. The data set is divided into two sets of data, the true positive and false positive, with each set also divided into the training, validation, and test sets for ML purposes.

### True Positive

This dataset was generated based on the synthetic objects inserted into the DEEP data by Pedro Bernardinelli, and then cutout from the survey dataset by Steven Stetzler. The original stamp cutouts are available on hyak at `/gscratch/dirac/DEEP/collab/fakes_cutouts/data/*/npy` (the `*` being the id of the fake object in DEEP).

### False Positive

This dataset was generated by running KBMOD on the DEEP data, but setting the set of serach angles to be roughly perpendicular to the ecliptic. This allows us to semi-safely assume that any search results are not real or inserted objects (i.e., while some real or synthetic objects may have been included in this set, the ratio to bad results is small enough to not be of concern for machine learning purposes).

### Data Transformation

Each source observation in the above set was ran through a series of random transformations, to enlarge the dataset and to capture all possible features of both sets. Those transformations were:
- select a random subset of the total set to coadd, with a minimum number of observations set to 25
    - so if the N number of observations is > 25 then the set of stamps used in coadding is of length [25,N]
- potentially offset the center pixel of each stamp by one in either direction, for both the X and Y in pixel space
    - so the set x_offset = [-1, 0, 1] and y_offset = [-1, 0, 1], creating nine potential offset states around the "actual" center of the object/results.
- potentially mirror around both the x and y axis
- potentially rotate 90 degrees around the center N times, where N is [0-3]

### Data Format

The data arrays are stored as numpy `.npy` files, and can be loaded with `x = np.load(PATH)`.

The shape of the data is `(N, 3, 21, 21)`, where N is the number of individual stamps in each set. The second dimmension corresponds to the type of coadd performed.
- index 0 = median coadd stamp
- index 1 = mean coadd stamp
- index 2 = sum coadd stamp.
Side note: the true positive stamp files also have an index 3, which corresponds to the recently added variance weighted coadd stamps. However, we found some discrepancies in those stamps and decided not to generate them for the false positive stamp. You can filter out this fourth column from the true positive set by slicing the array like [:,:3,:,:].

Each coadd stamp has the shape (21,21).

## Data

Here are the preloaded data sets. If you want to generate your own train, test, and validation sets, you can load in the `true_positive_stamps_full.npy` and `false_positive_stamps_full.npy` files.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/(POSGRADO) Universidad de Chile/ELECTIVE - Astroinformatic/Project

MessageError: Error: credential propagation was unsuccessful

In [ ]:
data_dir = "./kbmod_ml_stamps/"

# TRUE POSITVE PATHS
true_pos_full  = os.path.join(data_dir, "true_positive_stamps_full.npy")
true_pos_train = os.path.join(data_dir, "true_train_stamps.npy")
true_pos_valid = os.path.join(data_dir, "true_valid_stamps.npy")
true_pos_test  = os.path.join(data_dir, "true_test_stamps.npy")

# FALSE POSITVE PATHS
false_pos_full  = os.path.join(data_dir, "false_positive_stamps_full.npy")
false_pos_train = os.path.join(data_dir, "false_train_stamps.npy")
false_pos_valid = os.path.join(data_dir, "false_valid_stamps.npy")
false_pos_test  = os.path.join(data_dir, "false_test_stamps.npy")

In [ ]:
true_train = np.load(true_pos_train)[:,:3,:,:] # this indexing isn't strictly needed if you're just grabbing one of the other coadd type columns, see below cell.
true_test  = np.load(true_pos_test)[:,:3,:,:]

false_train = np.load(false_pos_train)
true_train.shape, false_train.shape

In [ ]:
true_train_median  = true_train[:,0,:,:]
false_train_median = false_train[:,0,:,:]

true_train_median.shape, false_train_median.shape

## Visualization

If you want to visualize what certain stamps look like, you can use `plt.imshow` and pass in the desired row.

Side note on visualizing the true positive stamps: we specifically weakened the signal on some of the observations to simulate a wider range of data. Visualizing some of the stamps might seem like mostly random noise, but I do promise there is signal there. Some of the time the PSF isn't visible in the median stamp but is seen in the mean or sum, or vice versa. I'm still messing around with the dataset so if we're running into too much low-SNR contimination in this set, we can raise the minimum observations in the selection algorithm and rerun everything, it only takes about 20 minutes to do :

In [ ]:
def plot_true_false(true, false):
    '''Plot a random set of true and false images'''
    fig_0,ax_0 = plt.subplots(nrows=2,ncols=5,figsize=[8,4])
    fig_1,ax_1 = plt.subplots(nrows=2,ncols=5,figsize=[8,4])
    ax_0 = ax_0.reshape(-1)
    ax_1 = ax_1.reshape(-1)
    true_int_list = np.random.choice(len(true),10,replace=False)
    false_int_list = np.random.choice(len(false),10,replace=False)
    for i, ax in enumerate(ax_0):
        ax.imshow(true[true_int_list[i]])
    for i, ax in enumerate(ax_1):
        ax.imshow(false[false_int_list[i]])
    fig_0.suptitle('True',fontsize=16)
    fig_1.suptitle('False',fontsize=16)
    fig_0.tight_layout()
    fig_1.tight_layout()

In [ ]:
plot_true_false(true_train[:,0,:,:], false_train[:,0,:,:])

## Training

I've included a simple model here as jumping off point. Go wild!

In [ ]:
false_pos_valid = os.path.join(data_dir, "false_valid_stamps.npy")
_a = np.load(false_pos_valid)

In [ ]:
# combine data
from tensorflow.keras.utils import to_categorical
def stack_data(f, t):
    input_stamps = np.vstack([f, t])
    stamp_class = np.zeros(len(f) + len(t))
    stamp_class[len(f):] = 1

    rand_state = np.random.RandomState(32)
    idx = rand_state.permutation(len(input_stamps))
    input_stamps = input_stamps[idx]
    stamp_class = stamp_class[idx]
    stamp_class = to_categorical(stamp_class)
    #stamp_class = stamp_class.astype(int)
    return input_stamps, stamp_class

In [ ]:
false_train_median.shape, true_train_median.shape

In [ ]:
true_valid_median  = np.load(true_pos_valid)[:,0,:,:]
false_valid_median = np.load(false_pos_valid)[:,0,:,:]

true_test_median = np.load(true_pos_test)[:,0,:,:]
false_test_median = np.load(false_pos_test)[:,0,:,:]

train_data, train_class = stack_data(false_train_median, true_train_median)
val_data, val_class     = stack_data(false_valid_median, true_valid_median)
test_data, test_class   = stack_data(false_test_median, true_test_median)

# Reshape [Channel, Heigh, Width]
train_data = train_data.reshape(-1,1,21,21)
val_data   = val_data.reshape(-1,1,21,21)
test_data  = test_data.reshape(-1,1,21,21)

In [ ]:
print('Train_data shape:',train_data.shape, 'Train_class shape:',train_class.shape)
print(train_class[:10])
print('Val_data shape:',val_data.shape,'Val_class shape:',val_class.shape)
print('Test_data shape:',test_data.shape,'Test_class shape:',test_class.shape)

In [ ]:
%%capture
try:
    import lightning as L
except:
    !pip install lightning

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("logs/")

import lightning as L
from torchsummary import summary
from torchmetrics.classification import Accuracy, Recall

from tqdm.auto import tqdm

device ='cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
initial_settings = {
    'batch_size': 512,
    'learning_rate': 1e-3,
}

[Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d)

Shape:
- Input:  $(N,C_{in},H_{in}, W_{in})$ or $(C_{in},H_{in}, W_{in})$
- Output: $(N,C_{out},H_{out}, W_{out})$ or $(C_{out},H_{out}, W_{out})$

Parameters:
- `in_channels`
- `out_channels`
- `kernel_size`
- `stride`: Default value is 1.
- `padding`: Default value is 0.
- `dilation`: Default value is 1.
- `groups`: controls the connections between inputs and outputs.
- `bias`: If `True`, adds a learnable bias to the output.
- `padding_mode`


$$ H_{out}=\left[\frac{H_{in}+2 \times \text{padding}[0]-\text{dilation}[0] \times (\text{kernel_size}[0]- 1)- 1}{\text{stride}[0]} +1 \right]
$$
$$ W_{out}=\left[\frac{W_{in}+2 \times \text{padding}[1]-\text{dilation}[1] \times (\text{kernel_size}[1]- 1)- 1}{\text{stride}[0]} +1 \right]
$$

Parameters:
- `kernel_size`
- `stride`: Default value is `kernel_size`.
- `padding`
- `dilation`
- `return_indices`: If `True`, will return te max indices along with the outputs. Useful for `torch.nn.MaxUnpool2d` later.
- `ceil_mode`: when `True`, will use _ceil_ instead of _floor_ to compute the output shape.


[`MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

Shape:
- Input:  $(N,C,H_{in}, W_{in})$ or $(C,H_{in}, W_{in})$
- Output: $(N,C,H_{out}, W_{out})$ or $(C,H_{out}, W_{out})$

Parameters:
- `kernel_size`
- `stride`: Default value is `kernel_size`.
- `padding`
- `dilation`
- `return_indices`: If `True`, will return te max indices along with the outputs. Useful for `torch.nn.MaxUnpool2d` later.
- `ceil_mode`: when `True`, will use _ceil_ instead of _floor_ to compute the output shape.

$$ H_{out}=\left[\frac{H_{in}+2 \times \text{padding}[0]-\text{dilation}[0] \times (\text{kernel_size}[0]- 1)- 1}{\text{stride}[0]} +1 \right]
$$
$$ W_{out}=\left[\frac{W_{in}+2 \times \text{padding}[1]-\text{dilation}[1] \times (\text{kernel_size}[1]- 1)- 1}{\text{stride}[0]} +1 \right]
$$

### Model V0: Basic

In [ ]:
class Basic(L.LightningModule):
    def __init__(self, n_classes):
        super().__init__()
        #self.save_hyperparameters()
        self.model_layers = nn.Sequential(
            # Input (Cin=1, Hin=21, Win=21, kenel_size=3, stride=1, padding=0, dilation=1)
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3),
            # Output (Cout=8, Hin=18, Win=18)
            nn.ReLU(),
            # Output (Cout=8, Hin=18, Win=18)
            nn.BatchNorm2d(num_features=8),
            # Output (Cout=8, Hin=18, Win=18)
            nn.MaxPool2d(kernel_size=2),
            # Output (Cout=8, Hin=9, Win=9)
            nn.Dropout(p=0.25),
            # Output (Cout=8, Hin=9, Win=9)
            nn.Flatten(),
            # Output (8*9*9)
            nn.Linear(in_features=8*9*9, out_features=64),
        )
        self.activation = nn.Sigmoid() if n_classes == 1 else nn.Softmax(dim=1)
        self.fc_out = nn.Linear(in_features=64, out_features=n_classes)

    def forward(self, x):
        # print(x.shape) # [Batch, Heigh, Width, Channel]
        x = x.view(x.size(0), 1, 21, 21)
        #x = x.permute(0,3,1,2) # # [Batch, Channel, Heigh, Width]
        x = self.model_layers(x)
        x = self.fc_out(x)
        x = self.activation(x)
        return x

model = Basic(n_classes=2).to(device) # Move the model to the GPU
summary(model, (21, 21, 1))  # Now call summary

### Model V1: ResNet-56

In [ ]:
class ResidualBlock(nn.Module):
    """

    change_size: flag to indicate if activations are within normal residual blocks
    """

    def __init__(self, in_channel, out_channel, stride, change_size=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1, stride=stride)
        self.bn1   = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1, stride=1)
        self.bn2   = nn.BatchNorm2d(out_channel)
        self.change_size = change_size
        # for changing activation map sizes
        if change_size:
            self.residual = nn.Sequential(nn.Conv2d(in_channel, out_channel,
                                            kernel_size=1, stride=stride),
                                nn.BatchNorm2d(out_channel)
            )

    def forward(self,x):
        identity = x if not self.change_size else self.residual(x)
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        y += identity

        return F.relu(y)


class ResNet56(nn.Module):
    """RestNet56
    """
    def __init__(self, n=9, num_classes=2):
        super().__init__()
        self.conv1  = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1    = nn.BatchNorm2d(16)
        self.block1 = self.create_block(n=9, in_channel=16, out_channel=16, stride=1, change_size=False)
        self.block2 = self.create_block(n=9, in_channel=16, out_channel=32, stride=2, change_size=True)
        self.block3 = self.create_block(n=9, in_channel=32, out_channel=64, stride=2, change_size=True)
        self.fc     = nn.Linear(64, num_classes)
        self.activation = nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)

    def create_block(self, n, in_channel, out_channel, stride, change_size=True):
        block = [ResidualBlock(in_channel, out_channel, stride, change_size)]
        for i in range(n-1):
            block.append(ResidualBlock(out_channel, out_channel, stride=1, change_size=False))
        return nn.Sequential(*block)

    def forward(self, x):
        x = x.view(x.size(0), 1, 21, 21)
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.block1(y)
        y = self.block2(y)
        y = self.block3(y)
        y = F.adaptive_avg_pool2d(input=y, output_size=1)
        y = self.fc(y.view(y.size(0), -1))
        y = self.activation(y)
        return y

model = ResNet56().to(device)   # Move the model to the GPU
summary(model, (21, 21, 1))  # Now call summary

### Model V2: ResNet-50

Here, model_parameters[‘resnet50’] = ([64,128,256,512],[3,4,6,3],4,True) represents the parameters for resnet-50 where

[64,128,256,512] -> channels in each intermediate block
[3,4,6,3] -> # repeatition for Bottlenecks in each block
4 -> expansion_factor. Note that 64 turns to 256, 128 to 512. All the resnet layers use the same expansion factor.
True -> create Bottleneck layer status. True only for ResNet 50+


In [ ]:
# This model is upload
import torchvision

class ResNet50(L.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet50 = torchvision.models.resnet50(weights="DEFAULT")

        # Dont change the parameters
        # Keep without change the parameters
        for param in self.resnet50.parameters():
            param.requires_grad = False

        self.resnet50.conv1 = nn.Conv2d(in_channels=1, out_channels=self.resnet50.conv1.out_channels,
                                        kernel_size=self.resnet50.conv1.kernel_size, stride=self.resnet50.conv1.stride,
                                        padding=self.resnet50.conv1.padding, bias=False) # change input channels to 21
        self.resnet50.fc = nn.Linear(in_features=self.resnet50.fc.in_features, out_features=num_classes)
        self.activation = nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)


    def forward(self, x):
        x = x.view(x.size(0), 1, 21, 21)
        y = self.resnet50(x)
        y = self.activation(y)
        return y

model = ResNet50(num_classes=2).to(device=device)
#summary(model, (1, 21, 21))

In [ ]:
x = torch.randn(5, 1, 21, 21).to(device=device)  # Batch de 5 imágenes
y_hat = model(x)
print(y_hat)

In [ ]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expansion, is_Bottleneck):
        super(Bottleneck, self).__init__()
        self.expansion = expansion
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.is_Bottleneck = is_Bottleneck

        if self.in_channels == self.out_channels*self.expansion:
            self.identity = True
        else:
            self.identity = False
            projection_layer = []
            projection_layer.append(nn.Conv2d(self.in_channels, self.out_channels*self.expansion, kernel_size=1, stride=stride))
            # Only Conv -> BN and no Relu
            # projection_layer.append(nn.ReLu())
            self.projection = nn.Sequential(*projection_layer)

        self.relu = nn.ReLU()

        # is_Bottleneck = True for all ResNet
        if self.is_Bottleneck:
            # Bottleneck
            # 1x1
            self.conv1 = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=1)
            self.bn1 = nn.BatchNorm2d(self.out_channels)
            # 3x3
            self.conv2 = nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=stride, padding=1)
            self.bn2 = nn.BatchNorm2d(self.out_channels)
            # 1x1
            self.conv3 = nn.Conv2d(self.out_channels, self.out_channels  * self.expansion, kernel_size=1, stride=1)
            self.bn3 = nn.BatchNorm2d(self.out_channels*self.expansion)

    def forward(self, x):
        identity = x
        if self.is_Bottleneck:
            # conv1x1 -> BN -> ReLU
            x = self.relu(self.bn1(self.conv1(x)))
            # conv3x3 -> BN -> ReLU
            x = self.relu(self.bn2(self.conv2(x)))
            # conv1x1 -> BN
            x = self.bn3(self.conv3(x))

        if self.identity:
            x += identity
        else:
            x += self.projection(identity)
        x = self.relu(x)
        return x

class ModelV2(nn.Module):
    """
    ResNet-50 model
    """

    def __init__(self, in_channels, num_classes):
        super(ModelV2, self).__init__()

        self.channels_list = [64,128,256,512]
        self.repeatition_list = [3,4,6,3]
        self.expansion = 4
        self.is_Bottleneck = True

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.block1 = self._make_blocks(64, self.channels_list[0], 3, stride=1, expansion=self.expansion, is_Bottleneck=self.is_Bottleneck)
        self.block2 = self._make_blocks(self.channels_list[0]*self.expansion, self.channels_list[1], self.repeatition_list[1], stride=2, expansion=self.expansion, is_Bottleneck=self.is_Bottleneck)
        self.block3 = self._make_blocks(self.channels_list[1]*self.expansion, self.channels_list[2], self.repeatition_list[2], stride=2, expansion=self.expansion, is_Bottleneck=self.is_Bottleneck)
        self.block4 = self._make_blocks(self.channels_list[2]*self.expansion, self.channels_list[3], self.repeatition_list[3], stride=2, expansion=self.expansion, is_Bottleneck=self.is_Bottleneck)

        self.average_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(self.channels_list[3]*self.expansion, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), 1, 21, 21)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)

        x = self.average_pool(x)

        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)

        return x

    def _make_blocks(self, in_channels, out_channels, num_repeat, stride, expansion, is_Bottleneck):
        layers = [Bottleneck(in_channels, out_channels, stride, expansion, is_Bottleneck)]
        for i in range(1, num_repeat):
            layers.append(Bottleneck(out_channels*expansion, out_channels, 1, expansion, is_Bottleneck))
        return nn.Sequential(*layers)

model = ModelV2(in_channels=1, num_classes=2).to(device)   # Move the model to the GPU
summary(model, (21, 21, 1))  # Now call summary

In [ ]:
class StampClassifier(L.LightningModule):
    def __init__(self, num_classes, model: str):
        super().__init__()

        if model == 'basic':
            self.model = Basic(num_classes)
        elif model == 'resnet56':
            self.model = ResNet56(num_classes)
        elif model == 'resnet50':
            self.model  = ResNet50(num_classes)

        # Metrics
        self.train_acc = Accuracy(task="multilabel", num_labels=num_classes)
        self.val_acc   = Accuracy(task="multilabel", num_labels=num_classes)
        self.test_acc  = Accuracy(task="multilabel", num_labels=num_classes)

        self.train_recall = Recall(task="multilabel", num_labels=num_classes)
        self.val_recall   = Recall(task="multilabel", num_labels=num_classes)
        self.test_recall  = Recall(task="multilabel", num_labels=num_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        #print('y:', y[:5])
        #print('y_hat:',y_hat[:5])

        loss = self.loss_function()(y_hat, y)

        # Log metrics
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", self.train_acc(y_hat, y), on_epoch=True, prog_bar=True)
        self.log("train_recall", self.train_recall(y_hat, y), on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_function()(y_hat, y)

        # Log metrics
        self.log("val_loss", loss, on_epoch=True)
        self.log("val_accuracy", self.val_acc(y_hat, y), on_epoch=True, prog_bar=True)
        self.log("val_recall", self.val_recall(y_hat, y), on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_function()(y_hat, y)
        self.log("test_loss", loss, on_epoch=True)
        self.log("test_accuracy", self.test_acc(y_hat, y), on_epoch=True)
        self.log("test_recall", self.test_recall(y_hat, y), on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=initial_settings['learning_rate'])
        return optimizer

    def loss_function(self):
        return nn.CrossEntropyLoss()

### Training and Validation

In [ ]:
class StampsDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data   = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label  = self.labels[idx]
        return sample, label

train_dataset      = StampsDataset(train_data, train_class)
validation_dataset = StampsDataset(val_data, val_class)
test_dataset       = StampsDataset(test_data, test_class)
train_dataloader   = DataLoader(train_dataset, batch_size=initial_settings['batch_size'], shuffle=True)
val_dataloader     = DataLoader(validation_dataset, batch_size=initial_settings['batch_size'], shuffle=False)
test_dataloader    = DataLoader(test_dataset, batch_size=initial_settings['batch_size'], shuffle=False)

In [ ]:
train_data.shape, train_class.shape, test_data.shape, test_class.shape

In [ ]:
x, y = next(iter(train_dataloader))
x.shape, y.shape, y[:5]

In [ ]:
x, y = next(iter(test_dataloader))
x.shape, y.shape, y[:10]

In [ ]:
from lightning.pytorch.callbacks import Callback

class MetricsCallback(Callback):
    def __init__(self):
        super().__init__()
        self.metrics = {
            "train_loss": [],
            "train_accuracy": [],
            "train_recall": [],
            "val_loss": [],
            "val_accuracy": [],
            "val_recall": [],
            "test_loss": [],
            "test_accuracy": [],
            "test_recall": [],
        }

    def on_train_epoch_end(self, trainer, pl_module):
        # Retrieve and save training metrics
        train_loss = trainer.callback_metrics.get("train_loss")
        train_accuracy = trainer.callback_metrics.get("train_accuracy")

        self.metrics["train_loss"].append(train_loss.item())
        self.metrics["train_accuracy"].append(train_accuracy.item())
        self.metrics["train_recall"].append(trainer.callback_metrics.get("train_recall").item())

    def on_validation_epoch_end(self, trainer, pl_module):
        # Retrieve and save validation metrics
        val_loss = trainer.callback_metrics.get("val_loss")
        val_accuracy = trainer.callback_metrics.get("val_accuracy")
        self.metrics["val_loss"].append(val_loss.item())
        self.metrics["val_accuracy"].append(val_accuracy.item())
        self.metrics["val_recall"].append(trainer.callback_metrics.get("val_recall").item())

    def on_test_epoch_end(self, trainer, pl_module):
        # Retrieve and save test metrics
        test_loss = trainer.callback_metrics.get("test_loss")
        test_accuracy = trainer.callback_metrics.get("test_accuracy")
        self.metrics["test_loss"].append(test_loss.item())
        self.metrics["test_accuracy"].append(test_accuracy.item())
        self.metrics["test_recall"].append(trainer.callback_metrics.get("test_recall").item())

    def get_metrics(self):
        """Returns the metrics dictionary for plotting."""
        return self.metrics

In [ ]:
models = ['basic', 'resnet56', 'resnet50']
#models = ['resnet56']#, 'resnet50']
results_metrics = {}
results_predictions = {}

for model in models:
    classifier = StampClassifier(num_classes=2, model=model)
    metrics_callback = MetricsCallback()

    trainer = L.Trainer(max_epochs=20, callbacks=[metrics_callback])
    trainer.fit(classifier, train_dataloader, val_dataloader)

    # Retrieve metrics
    metrics = metrics_callback.get_metrics()

    # Store the results
    results_metrics[model] = metrics

    # Test the model and save predictions
    probabilities = []
    predictions   = []
    true_labels   = []

    classifier.eval()
    with torch.inference_mode():
        for batch in test_dataloader:
            x, y = batch
            y_hat = classifier(x)
            preds = (y_hat >= 0.5).float()
            probabilities.extend(y_hat.cpu().numpy())
            predictions.extend(preds.cpu().numpy())  # Almacena las predicciones binarizadas
            true_labels.extend(y.cpu().numpy())

    # Convertir etiquetas y predicciones a índices solo para la matriz de confusión
    true_labels_indices = [np.argmax(label) for label in true_labels]  # `[1, 0]` → `0`, `[0, 1]` → `1`
    predictions_indices = [np.argmax(pred) for pred in predictions]  # `[1, 0]` → `0`, `[0, 1]` → `1`

    # Calcular la matriz de confusión
    cm = confusion_matrix(true_labels_indices, predictions_indices, labels=[0, 1])

    # Save predictions and true labels for further analysis
    results_predictions[model] = {
        'probabilities': probabilities,
        "predictions": predictions,
        "true_labels": true_labels,
        "confusion_matrix": cm,
    }

In [ ]:
for _, (model, metrics) in enumerate(results_metrics.items()):
    print(f"Model: {model}")
    print(f"Train Loss: {np.round(metrics['train_loss'],decimals=3)}")
    print(f"Train Accuracy: {np.round(metrics['train_accuracy'],decimals=3)}")
    print(f"Train Recall: {np.round(metrics['train_recall'],decimals=3)}")
    print(f"Validation Loss: {np.round(metrics['val_loss'],decimals=3)}")
    print(f"Validation Accuracy: {np.round(metrics['val_accuracy'],decimals=3)}")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=[10, 10])
fig.subplots_adjust(hspace=0.5)
for _, (model, metrics) in enumerate(results_metrics.items()):
    #ls = {'basic': 'solid', 'resnet56': 'dashed', 'resnet50': 'dotted'}[model]
    color = {'basic': 'C0', 'resnet56': 'C1', 'resnet50': 'C2'}[model]
    ax[0].plot(metrics['train_loss'], color=color, label=f'Training Loss - {model}')
    ax[0].plot(metrics['val_loss'], color=color, ls='dashed', label=f'Validation Loss - {model}')

    ax[1].plot(metrics['train_accuracy'], color=color, label=f'Training Accuracy - {model}')
    ax[1].plot(metrics['val_accuracy'], color=color, ls='dashed', label=f'Validation Accuracy - {model}')

    ax[2].plot(metrics['train_recall'], color=color, label=f'Training Recall - {model}')
    ax[2].plot(metrics['val_recall'], color=color, ls='dashed', label=f'Validation Recall - {model}')

ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].set_title('Loss Curves')
ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))  # optionally show legend

ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy')
ax[1].set_title('Accuracy Curves')
ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))  # optionally show legend

ax[2].set_xlabel('Epoch')
ax[2].set_ylabel('Recall')
ax[2].set_title('Recall Curves')
ax[2].legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig('./loss_accuracy.png', dpi=300)
plt.show()       # display the plot

In [ ]:
len(results_predictions['basic']), results_predictions['basic'].keys(), len(results_predictions['basic']['probabilities'])

In [ ]:
results_predictions['basic']['predictions'][:10]

In [ ]:
n = 20
for batch in test_dataloader:
    x, y = batch
    test_images = x[:n,...]
    true_labels = y[:n,...]
    break

num_cols = 5
num_rows = (n + num_cols - 1) // num_cols  # Compute rows dynamically

for model in models:
    fig, ax = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=[16, 12])
    fig.suptitle(f'Test Images Prob Model:{model}')
    ax = ax.flatten()

    # Retrieve the probabilities for the current model
    probabilities = np.array(results_predictions[model]['probabilities'][:n])
    predictions = (probabilities >= 0.5).astype(int)  # Binarize predictions

    #print(probabilities)

    # Remove unused axes
    for tmp_ax in ax[len(test_images[:n]):]:
        tmp_ax.set_axis_off()

    for i, image in enumerate(test_images[:n]):
        is_correct = np.array_equal(predictions[i], true_labels[i].cpu().numpy())
        title_color = "green" if is_correct else "red"
        ax[i].imshow(image[0], cmap="viridis")  # Use channel 0 for grayscale
        ax[i].set_title(
            f"Pred: {predictions[i].tolist()} (True: {[int(x) for x in true_labels[i].tolist()]})\nProb: {probabilities[i,1]:.2f}",
            color=title_color,
        )
        ax[i].set_axis_on()

    plt.tight_layout()
    plt.savefig(f'./test_images_prob_{model}.png', dpi=300)  # Use channel 0 for grayscale
    plt.show()

In [ ]:
# Optionally, plot the confusion matrix
for model in models:
    cm = results_predictions[model]['confusion_matrix']
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["FP", "TP"])
    disp.plot(cmap="Blues")
    plt.title(f"Confusion Matrix for {model}")
    plt.savefig(f'./confusion_matrix_{model}.png', dpi=300)
    plt.show()

below 1sigma or 2sigma

# Keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K

In [ ]:
def simple_model(input_shape=(21,21,1), n_classes: int = 2):
    model = tf.keras.models.Sequential(name='simple')
    model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=input_shape, name='conv1'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn1'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu', name='fc_1'))
    activation = 'sigmoid' if n_classes == 1 else 'softmax'
    model.add(tf.keras.layers.Dense(n_classes, activation=activation, name='fc_out'))
    return(model)

In [ ]:
simple_cnn.summary()

In [ ]:
simple_cnn = simple_model(input_shape=(21,21,1))
sgd = tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.05, nesterov=True)
simple_cnn.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
n_epochs = 20
simple_model_history = simple_cnn.fit(train_data, train_class, epochs=n_epochs, batch_size=512, verbose=1, validation_data=(val_data, val_class), shuffle=True)

In [ ]:
model_history = simple_model_history
import matplotlib.pyplot as plt
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
plt.figure()
plt.plot(np.arange(n_epochs), loss, label='Training Loss')
plt.plot(np.arange(n_epochs), val_loss, label='Validation Loss')
plt.title('Loss Curves')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.savefig('Keras_Loss.png')
plt.figure()
plt.plot(np.arange(n_epochs), acc, label='Training Accuracy')
plt.plot(np.arange(n_epochs), val_acc, label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.savefig('figures/ResNet50_LC.png')